In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd

from dhdrnet.util import DATA_DIR, ROOT_DIR

figdir = ROOT_DIR / "figures"

In [3]:
df = (
    pd.read_csv(
        ROOT_DIR / "precomputed_data" / "store_test_2020-10-06.csv", index_col=0
    )
    .drop(columns="ev1")
    .rename(columns={"ev2": "ev"})
)
df

,name,metric,ev,score
0,c1b1_20150312_141022_171,rmse,-3.00,0.813147
1,c1b1_20150312_141022_171,psnr,-3.00,6.753222
2,c1b1_20150312_141022_171,ssim,-3.00,0.242182
3,c1b1_20150312_141022_171,rmse,-2.75,0.814743
4,c1b1_20150312_141022_171,psnr,-2.75,6.736195
...,...,...,...,...
103,4KK2_20150829_170525_488,psnr,5.75,14.610017
104,4KK2_20150829_170525_488,ssim,5.75,0.835229
105,4KK2_20150829_170525_488,rmse,6.00,0.318225
106,4KK2_20150829_170525_488,psnr,6.00,13.930960


In [4]:
grouped = df.groupby("metric")
df_rmse = grouped.get_group("rmse").drop(columns="metric")
df_psnr = grouped.get_group("psnr").drop(columns="metric")
df_ssim = grouped.get_group("ssim").drop(columns="metric")

In [5]:
df_rmse

,name,ev,score
0,c1b1_20150312_141022_171,-3.00,0.813147
3,c1b1_20150312_141022_171,-2.75,0.814743
6,c1b1_20150312_141022_171,-2.50,0.817379
9,c1b1_20150312_141022_171,-2.25,0.819293
12,c1b1_20150312_141022_171,-2.00,0.822532
...,...,...,...
93,4KK2_20150829_170525_488,5.00,0.194159
96,4KK2_20150829_170525_488,5.25,0.229136
99,4KK2_20150829_170525_488,5.50,0.263132
102,4KK2_20150829_170525_488,5.75,0.294294


In [6]:
dfg = df.groupby(["name", "metric"])
ind_minscore = dfg.idxmin()["score"]
ind_minscore

name                      metric
0006_20160721_163256_525  psnr      52
                          rmse      84
                          ssim      11
0006_20160721_172953_515  psnr       7
                          rmse      84
                                    ..
c483_20150901_211301_691  rmse      72
                          ssim      11
c483_20150901_211303_855  psnr      10
                          rmse      69
                          ssim      14
Name: score, Length: 8685, dtype: int64

In [16]:
dfg.min()["score"].groupby("metric").describe()

,count,mean,std,min,25%,50%,75%,max
metric,,,,,,,,
psnr,2895.0,11.401944,2.976953,5.981377,8.948695,10.805977,13.435982,21.903930
rmse,2895.0,0.078918,0.030414,0.010718,0.056968,0.072008,0.093795,0.236306
ssim,2895.0,0.517660,0.264114,0.014725,0.296821,0.522967,0.743614,0.970097


In [9]:
ind_minscore["5066_20160416_100212_436"]

metric
psnr    16
rmse    72
ssim    17
Name: score, dtype: int64

In [20]:
df2 = pd.read_csv(ROOT_DIR / "precomputed_data" / "store_no_synth.csv")
df2 = df2[(df2["ev1"] == 0) | (df2["ev2"] == 0)]

In [21]:
df2["ev"] = df2[["ev1", "ev2"]].apply(
    lambda evs: [e for e in evs if e != 0][0], axis=1
)
df2 = df2.drop(columns=["ev1", "ev2"])
df2

,name,metric,score,ev
33,0009_20160703_160626_325,mse,344.949785,-3.00
34,0009_20160703_160626_325,ssim,0.951258,-3.00
35,0009_20160703_160626_325,perceptual,0.049351,-3.00
117,0009_20160703_160626_325,mse,353.442578,-2.75
118,0009_20160703_160626_325,ssim,0.951578,-2.75
...,...,...,...,...
7230337,33TJ_20150730_201700_327,ssim,0.808433,5.75
7230338,33TJ_20150730_201700_327,perceptual,0.161453,5.75
7230339,33TJ_20150730_201700_327,mse,3017.795220,6.00
7230340,33TJ_20150730_201700_327,ssim,0.789549,6.00


In [32]:
import numpy as np
rmse = df2[df2["metric"] == "mse"]
rmse = rmse["score"].apply(np.sqrt)
rmse.max()

131.68803635158815

In [33]:
df[df["metric"] == "rmse"]

name      c483_20150901_211303_855
metric                        rmse
ev                               6
score                      1.14268
dtype: object